In [ ]:
#backtest
state = bool
state_temp = False

for x in range(interval_2+1, graph_interval):
    state = ma_50_result[x] > ma_200_result[x] # TRUE = LONG, FALSE = SHORT
    if state != state_temp:
        state_temp = state
        if state:
            print("Buy")
        else:
            print("Sell")
    else:
        print("No change in state")


In [2]:
import telegram.ext
from telegram.ext import Updater, CommandHandler
from telegram.ext import MessageHandler, Filters
import pandas as pd
import numpy as np
from binance.client import Client
import os
import matplotlib.pyplot as plt
import statistics
from binance.enums import *
from decimal import Decimal
import datetime
# new_trade = trade("BINANCE_KEY","BINANCE_SECRET")
# new_trade.getHistory(symbol,interval)
# binance setup
binance_api_key = os.environ.get("BINANCE_KEY")
binance_api_secret = os.environ.get("BINANCE_SECRET")
client = Client(binance_api_key, binance_api_secret)
# telegram setup

#for kline in client.get_historical_klines_generator("BTCUSDT","1d","1 Jan, 2019"):

def backTest (ma_1_interval, ma_2_interval,interval: str):

    history = client.get_historical_klines("BTCUSDT",interval,"1 Jan, 2019")
    for x in range(len(history)):
        history[x][4] = float(history[x][4])
    
    ma_1_interval = ma_1_interval
    ma_2_interval = ma_2_interval
    bollinger_interval = 9
    budget_usd = 100
    budget_btc = 0
    
    max_interval = max(ma_1_interval,ma_2_interval)
    
    count = 0
    
    for x in range(len(history)):
        count = count + 1
        history_ma_1 = []
        history_ma_2 = []
        for x in range(ma_1_interval-1):
            history_ma_1.append(float(history[count-ma_1_interval+x][4]))
        for x in range(ma_2_interval-1):
            history_ma_2.append(float(history[count-ma_2_interval+x][4]))
        while count > max_interval and count < len(history)-1:
            ma_1 = np.mean(history_ma_1)
            ma_2 = np.mean(history_ma_2)
            state = ma_1 > ma_2
            #print(state)
            if state:
                buy_price = float(history[count][4])
                if budget_btc == 0:
                    budget_btc = budget_usd / buy_price
                else:
                    #print("btc",budget_btc, "price", buy_price)
                    budget_usd = 0
            else:
                sell_price = float(history[count][4])
                if budget_usd == 0:
                    budget_usd = budget_btc * sell_price
                else:
                    #print("usd",budget_usd, "price", sell_price)
                    budget_btc = 0
            break
    total_eff = (budget_btc * float(history[-1][4]) + budget_usd) / 100
    #print(total_eff)
    market_eff = (float(history[-1][4]) - float(history[0][4]))/float(history[0][4])
    #print(market_eff)
    #print("market grew by",(market_eff)*100, "%")
    bot_eff = total_eff - market_eff
    total_budget = budget_btc * float(history[-1][4]) + budget_usd
    #print(total_budget, bot_eff)
    return total_budget

def getKeysByValue(dictOfElements, valueToFind):
    listOfKeys = list()
    listOfItems = dictOfElements.items()
    for item  in listOfItems:
        if item[1] == valueToFind:
            listOfKeys.append(item[0])
    return  listOfKeys

In [10]:
backTest(5,7,"15m")

192.3954154057589

In [1]:
#mylist = []
mydict = {}
count = 0
intervals = ['1m','3m','5m','15m','30m','1h','2h','4h','6h','8h','1d','3d','1w','1M']
intervals = intervals[:3]
print(intervals)

for interval in intervals:
    for ma1 in range(5,10):
        for ma2 in range(5,10):
            count = count + 1
            print(count)
            print(interval)
            #mylist.append(backTest(i,j))
            mydict["ma1 - {0}, ma2 - {1}".format(ma1,ma2)] = backTest(ma1,ma2,interval)
            #print(count)

maximum_value = max(mydict.values())
maximum_key = max(mydict.keys(), key=(lambda key: mydict[key]))   
#print(getKeysByValue(mydict, max(mydict)))
print(maximum_value)
print(maximum_key)
#print(mydict)

['1m', '3m', '5m']
1
1m


NameError: name 'backTest' is not defined